In [1]:
import sagemaker  # for built in algos
import boto3  # reading s3 buckets from public DNS
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

In [12]:
bucket_name = "s3testapplication"
my_region = boto3.session.Session().region_name
print(my_region)

us-east-2


In [13]:
s3 = boto3.resource('s3')
try:
    if my_region=='us-east-2':
        s3.create_bucket(Bucket=bucket_name)
    else: 
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print("S3 error: ", e)

S3 bucket created


In [14]:
# set the output path for model output
prefix = "built-in-xgb"  # just a folder
out_path = "s3://{}/{}/output".format(bucket_name, prefix)
print(out_path)

s3://s3testapplication/built-in-xgb/output


In [15]:
import pandas as pd
import urllib

try:
    urllib.request.urlretrieve("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [16]:
model_data.head()

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,marital_unknown,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,default_no,default_unknown,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
2,37,1,999,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
3,40,1,999,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
4,56,1,999,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0


In [17]:
model_data.shape

(41188, 61)

In [18]:
# train test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=4), [int(0.7*model_data.shape[0])])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [23]:
# saving train and test set to s3 bucket
import os

pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


In [24]:
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.s3_input(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


Built in XGBoost

In [25]:
# models present in containers
container = get_image_uri(boto3.Session().region_name,
                         'xgboost',
                         repo_version='1.0-1')

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


In [29]:
# initialize hyperparams
hyperparameters = {
    'max_depth':'5',
    'eta':'0.2',
    'gamma':'4',
    'min_child_weight':'6',
    'subsample':'0.8',
    'silent':'0',
    'objective':'binary:logistic',
    'num_round':'100'
}

In [30]:
# construct a sagemaker estimator that calls the xgb container
estimator = sagemaker.estimator.Estimator(image_name=container,
                                         hyperparameters=hyperparameters,
                                         role=sagemaker.get_execution_role(),
                                         train_instance_count=1,
                                          train_instance_type='ml.m5.2xlarge',
                                          train_volume_size=5,
                                          output_path=out_path,
                                          train_use_spot_instances=True,
                                          train_max_run=300,
                                          train_max_wait=600
                                         )

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [31]:
estimator.fit({'train':s3_input_train, 'validation':s3_input_test})

2020-09-18 12:10:30 Starting - Starting the training job...
2020-09-18 12:10:33 Starting - Launching requested ML instances......
2020-09-18 12:11:38 Starting - Preparing the instances for training...
2020-09-18 12:12:23 Downloading - Downloading input data...
2020-09-18 12:12:52 Training - Downloading the training image..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[12:13:09] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
INFO:root:Determined delimiter of 

In [32]:
xgb_predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


---------------!

In [33]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no','y_yes'], axis=1).values
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
predictions = xgb_predictor.predict(test_data_array).decode('utf-8')
predictions_array = np.fromstring(predictions[1:], sep=',')
print(predictions_array.shape)

(12357,)


In [34]:
# perfromance metrics
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.8%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10793)    38% (190)
Purchase        9% (1070)     62% (304) 



In [35]:
#### deleting endpoints
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '5D4722C7599BF208',
   'HostId': 'yzS0XnzBiYjdK5ip1qhWA9XeLWIkINkdDWQ9iegi6afFCIzPfs3R+5/VaMllrgpHep44vfvmNNw=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'yzS0XnzBiYjdK5ip1qhWA9XeLWIkINkdDWQ9iegi6afFCIzPfs3R+5/VaMllrgpHep44vfvmNNw=',
    'x-amz-request-id': '5D4722C7599BF208',
    'date': 'Fri, 18 Sep 2020 12:30:17 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'built-in-xgb/output/sagemaker-xgboost-2020-09-18-12-10-30-755/output/model.tar.gz'},
   {'Key': 'built-in-xgb/train/train.csv'},
   {'Key': 'built-in-xgb/test/test.csv'}]}]